This is a retrospective data analysis of our ESPN Fantasy Football league data at the end of Week 17 for the 2022 season. The code can be re-used for any past season using the same ESPN API.

I used the [Pro Football Reference](https://www.pro-football-reference.com/years/2022/games.htm) for NFL schedules and [Christian Wendt's ESPN Fantasy Football API](https://github.com/cwendt94/espn-api) to extract ESPN league-specific data.

For data visualization, I used the [Altair Data Visualization Library](https://altair-viz.github.io/). Altair was chosen for its interactivity, especially for its ability to display hover data and the compatibility of interactive plots with GitHub Pages.

One question this analysis can answer is: what was the return on investment of a given players with fantasy draft or waiver budget dollars (FAAB).

See src directory for source code.

In [1]:
%cd /Users/jonathancheng/PycharmProjects/espnff/src/

import nfl_schedule as nf
import ff_league_data as ff
from plotting import scatterplot_acquisitions


/Users/jonathancheng/PycharmProjects/espnff/src


## Get NFL Schedule

In [2]:
year_of_interest=2022

path = r'/Users/jonathancheng/PycharmProjects/espnff/data'

league_id = 1094090
year = year_of_interest

swid = "{F191FB8C-DB2D-4D24-91FB-8CDB2DED249D}"
s2='AECJMQHsUHB0FTXdZkw93uY7GRbX8BPnm93Ye6AwvwrMsrZFGg1Lbmi07SWVov2ioN8zGMFDzZiiDSeQCa7WQHaGivGnMfGWLjmfGwkOeLXb5baD1sltp%2B%2BIfHAtl98TpmHgB16ZpGn6g3Bm5vLEA7yDC6HkbD3LSp0E2rGB7hKziLMvZ7mT6ONJFRe8Xp3ApYWSvxPr9cz0pJiI%2FF0blsZ8hyATDJMEyaQ2O%2FypcsViORr6hqYTmXHPuPKnMBfvYC8LQqi1exGw3vnyg6ptsB2Y'

espn_s2 = s2


In [3]:
df_proteam_schedule = nf.get_nfl_schedule(year_of_interest)
season_start_date = nf.get_season_start_date(df_proteam_schedule)

## Generate League object

In [4]:
league = ff.fetch_espn_api(league_id, year, espn_s2, swid)
activity_ls = league.recent_activity(1000000)
wk_ls = ff.get_weeks(league)

## Get Acquisitions Data

In [5]:
# fetch league data, wrangle into acquisitions DataFrame

acq_data_flat_ls = ff.get_acq_ls(activity_ls)

df_acq = ff.build_df_acq(acq_data_flat_ls)

## Get Draft Data

In [6]:
df_draft,drafted_players = ff.build_df_draft(league)

## Get total points of rostered players Dataframe

In [7]:
df_rostered = ff.build_df_rostered(league)

## Get total points of free agent players Dataframe

In [8]:
df_FA = ff.build_df_FA(league)

In [9]:
# Generate all player stats dataframe, including all Free Agents
df_player_stats = ff.build_df_player_stats(df_rostered,df_FA)

In [10]:
df_draft_stats = ff.build_df_draft_stats(df_draft,df_player_stats)
df_acq_stats = ff.build_df_acq_stats(df_acq,df_player_stats)
df_acq_final = ff.build_df_acq_final(season_start_date, df_draft_stats, df_acq_stats, drafted_players)

## Get player_box_scores from fantasy season

In [11]:
df_player_box_scores = ff.build_df_player_box_scores(league, wk_ls)

## Construct df_stints

In [12]:
df_stints=ff.build_df_stints(df_acq_final, df_proteam_schedule, df_player_stats, drafted_players)

In [13]:

df_stints['Total points per stint'] = df_stints.apply(lambda x: ff.get_total_pts_per_player(x['Player'], x['Stint (wks)'], df_player_box_scores),axis=1).fillna(0)

In [14]:
df_stints.head()

,Stint_id,Player,Team,ProTeam,Added,Bid Amount ($),Dropped,Stint (wks),Position,Drafted,Total points per stint
0,0,49ers D/ST,Big Joshy Style,SF,2022-09-07,1,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",D/ST,True,140.0
1,1,A.J. Brown,Ice City USA,PHI,2022-09-07,34,NaT,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",WR,True,244.1
2,2,AJ Dillon,The Genaissance,GB,2022-09-07,4,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,150.3
3,3,Aaron Jones,Kirk-life Balance,GB,2022-09-07,42,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,212.8
4,4,Aaron Rodgers,Frankel's Cankles,GB,2022-09-07,3,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",QB,True,226.6


Running Backs in the Draft

In [15]:
plot_title,chart = scatterplot_acquisitions(df_stints, position='RB', acq_by_draft=True)
chart

alt.Chart(...)

Wide Receivers in the Draft

In [16]:
plot_title,chart = scatterplot_acquisitions(df_stints, position='WR', acq_by_draft=True)
chart

alt.Chart(...)

## Waiver Wire Acquisitions

In [17]:
plot_title,chart = scatterplot_acquisitions(df_stints, position='WR', acq_by_draft=False)
chart

alt.Chart(...)

Curtis Samuel was the most valuable waiver wire receiver.